In [ ]:
# this draft code is old and inaccurate, as simpleHierarchy model should be used differently (correct usage found in model_v03.. ipynb file)

cgs = { # cgs <==> class groups
        "colorDiversity": ("colorDiversity", 2), # many or few
        "manyColors": ("manyColors", 4), # selfies, memes (e or f), or eGreetingsAndMisc (+1 if none of them)
        "fewColors": ("fewColors", 4), # socialMedia (e or f), fTxtMssgs, academic (photos or digital) (+1 if none of them)
        "memes": ("memes", 3), # e or f (+1 if none of them)
        "socialMedia": ("socialMedia", 3), # e or f (+1 if none of them)
        "academic": ("academic", 3) # photos or academic (+1 if none of them)
    }

cs = { # cs <==> classes
    "selfies": ("selfies", 1),
    "fmemes": ("fmemes", 1),
    "ememes": ("ememes", 1),
    "fSocialMedia": ("fSocialMedia", 1),
    "eSocialMedia": ("eSocialMedia", 1),
    "fTxtMssgs": ("fTxtMssgs", 1),
    "eGreetingsAndMisc": ("eGreetingsAndMisc", 1),
    "academicPhotos": ("academicPhotos", 1),
    "academicDigital": ("academicDigital", 1),
}

# smaller hierarchy
hierarchy = {
    cgs["colorDiversity"] : [cgs["manyColors"], cgs["fewColors"]],
    cgs["manyColors"] : [cgs["memes"]],
    cgs["fewColors"] : [cgs["socialMedia"], cgs["academic"]],
}

# mapping flat labels to hierarchical labels
labelToHierarchy = {
    0 : [
            0, # diversity of colors: many or few (so classes: 0,1)
            0, # many colors: selfies, memes (fmm or emm), or eGreetingsAndMisc (+1 if none of them) (so classes: 0,1,2,3)
            3, # few colors: socialMedia (fsm or esm), fTxtMssgs, academic (photos or digital) (+1 if none of them) (so classes: 0,1,2,3)
            2, # meme type: fmm or emm (+1 if none of them) (so classes: 0,1,2)
            2, # social media type: fsm or esm (+1 if none of them) (so classes: 0,1,2)
            2, # academic type: photos or digital (+1 if none of them) (so classes: 0,1,2)
        ], 
    1 : [0, 1, 3, 0, 2, 2],
    2 : [0, 1, 3, 1, 2, 2],
    3 : [1, 3, 0, 2, 0, 2],
    4 : [1, 3, 0, 2, 1, 2],
    5 : [1, 3, 1, 2, 2, 2],
    6 : [0, 2, 3, 2, 2, 2],
    7 : [1, 3, 2, 2, 2, 0],
    8 : [1, 3, 2, 2, 2, 1],
}

# mapping hierarchical labels (integers) to hierarchical names (strings)
hierLabelToClassName = {
    # diversity of colors: many or few (so classes: 0,1)
    0 : ['manyColors', 'fewColors'], 
    # many colors: selfies, memes (fmm or emm), or eGreetingsAndMisc (+1 if none of them) (so classes: 0,1,2,3)
    1 : ['00. selfies', 'memes', '70. eGreetingAndMisc', 'none'],
    # few colors: socialMedia (fsm or esm), fTxtMssgs, academic (photos or digital) (+1 if none of them) (so classes: 0,1,2,3)
    2 : ['socialMedia', '50. fTxtMssgs', 'academic', 'none'],
    # meme type: fmm or emm (+1 if none of them) (so classes: 0,1,2)
    3 : ['10. fmemes', '20. ememes', 'none'],
    # social media type: fsm or esm (+1 if none of them) (so classes: 0,1,2)
    4 : ['30. fSocialMedia', '40. fSocialMedia', 'none'],
     # academic type: photos or digital (+1 if none of them) (so classes: 0,1,2)
    5 : ['81. academicPhotos', '82. academicDigital', 'none'],
}

# cgs = { # cgs <==> class groups
#         "colorDiversity": ("colorDiversity", 2), # many or few
#         "manyColors": ("manyColors", 4), # selfies, memes (e or f), or eGreetingsAndMisc
#         "fewColors": ("fewColors", 5), # socialMedia (e or f), fTxtMssgs, academic (photos or digital)
#         "memes": ("memes", 2), # e or f
#         "socialMedia": ("socialMedia", 2), # e or f
#         "academic": ("academic", 2) # photos or academic
#     }

# # bigger hierarchy (I think you shouldn't use classes which have "1" as a value for the class)
# hierarchy = {
#     cgs["colorDiversity"] : [cgs["manyColors"], cgs["fewColors"]],
#     cgs["manyColors"] : [cs["selfies"], cgs["memes"], cs["eGreetingsAndMisc"]],
#     cgs["fewColors"] : [cgs["socialMedia"], cs["fTxtMssgs"], cgs["academic"]],

#     cgs["memes"] : [cs["fmemes"], cs["ememes"]],
#     cgs["socialMedia"] : [cs["fSocialMedia"], cs["eSocialMedia"]],
#     cgs["academic"] : [cs["academicPhotos"], cs["academicDigital"]],
# }


In [ ]:
def create_hnn_model_arch(model_version:str, img_size:tuple, class_groups:dict, classes:dict, hierarchy:dict):
    base_model = nn.Sequential(
        # note that padding is 0 to re-produce TF's implementation of m01, where the default value for 'padding' argument is 'valid' which means no padding
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0), 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        nn.Flatten(),
        # deviated from TF implementation of m01 by making this 1024 instead of 512
        nn.LazyLinear(1024),
        nn.ReLU(),
        # unlike TF implementation of m01, we'll not include this layer here, rather we'll add it later
        # nn.Softmax(dim=1) 
    )

    # these are the indepdent layers of parent and children
    model_layers = [
        # the output of this layer is feed forward from parent to child
        nn.Linear(1024, 512),
        nn.Linear(512, 256),
        nn.Linear(256, 128),
        nn.Linear(128, 64),
        # we do not include the layer below, as it is automatically created at the end of each parent/child. check this for details:
        # https://github.com/rajivsarvepalli/SimpleHierarchy/blob/8e4c29f334928f43509b2d328b11b4a83f2d2af6/src/simple_hierarchy/hierarchal_model.py#L173
        # nn.Linear(64, num_classes), 
    ]

    # 1024 is the output size of our base model
    # 1024 is the input size of our additional indepdent layers (called model_layers)
    # 64 is the output size of our additional indepdent layers (called model_layers) (excluding the final automatically-created hidden layer)
    # 512 is the output size of fourth to last additional indepdent layer to feed (note: we say 'third to last' if we neglect talking about the final automatically-created hidden layer)
    # forward from parent to child (with concatenation)
    size = (1024,1024,64,512)
    # all 4 layers are distinct for each grouping of classes of model_layers (actually 5 layers if you add the automatically-created hidden layer)
    k = 4
    # we want to feed from the fourth to last layer (from parent to child (with concatenation)) (note: we say 'third to last' if we neglect talking about the final automatically-created hidden layer)
    feed_from = 3
    output_order = [*class_groups.values()]
    idx_to_class_name = {k : v for k, v in enumerate([*class_groups.keys(), *classes.keys()])}
    model = DemoModel(hierarchy, base_model, size, model_layers, k, feed_from, output_order)
    
    a = torch.rand(3,*img_size).unsqueeze(0)
    # running an arbitrary forward pass to initialze weights/params (since LazyLinear was used)
    model(a) 

    writer = SummaryWriter(f"./models/hnn_model_v{model_version}_tb_graphs")
    writer.add_graph(model, a)
    writer.close()

    return model, idx_to_class_name
# Example input
img_size = (306,306)
demo_model, idx_to_class_name = create_hnn_model_arch("03", img_size, cgs, cs, hierarchy)
a = torch.rand(3,*img_size).unsqueeze(0)
demo_model(a)
